In [1]:
import numpy as np
import pandas as pd
import pickle                  # serialization stdlib
import sklearn.preprocessing

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [6]:
# Check kernel.json references generic "python3", not sys.path.python3
# !pip freeze  pandas=1.1.5
pd.__version__

'1.1.5'

In [2]:
brazil_df = pd.read_excel("dataset.xlsx")
brazil_df

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,ae66feb9e4dc3a0,3,positive,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5640,517c2834024f3ea,17,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,5c57d6037fe266d,4,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5642,c20c44766f28291,10,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
brazil_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Columns: 111 entries, Patient ID to ctO2 (arterial blood gas analysis)
dtypes: float64(70), int64(4), object(37)
memory usage: 4.8+ MB


In [16]:
# Checking null counts
pd.set_option('display.max_rows', None)
null_count = brazil_df.isnull().sum().sort_values()
null_count

Patient ID                                                  0
Patient age quantile                                        0
SARS-Cov-2 exam result                                      0
Patient addmited to regular ward (1=yes, 0=no)              0
Patient addmited to semi-intensive unit (1=yes, 0=no)       0
Patient addmited to intensive care unit (1=yes, 0=no)       0
Influenza B                                              4290
Respiratory Syncytial Virus                              4290
Influenza A                                              4290
Rhinovirus/Enterovirus                                   4292
Inf A H1N1 2009                                          4292
CoronavirusOC43                                          4292
Coronavirus229E                                          4292
Parainfluenza 4                                          4292
Adenovirus                                               4292
Chlamydophila pneumoniae                                 4292
Parainfl

In [21]:
# Cutoff for not enough data... One way of looking at things

threshold = 0.80
null_cutoff = threshold * brazil_df.shape[0]

print(f"Null threshold: {threshold}%, Null count: {null_cutoff}")
print(null_count[null_count > null_cutoff].count())
                 
# Convert the list of attributes with too many nulls to a list
attr_deletions = null_count[null_count > null_cutoff].index

attr_deletions


Null threshold: 0.8%, Null count: 4515.2
88


Index(['Influenza A, rapid test', 'Influenza B, rapid test', 'Hemoglobin',
       'Hematocrit', 'Red blood cell distribution width (RDW)', 'Platelets',
       'Mean corpuscular volume (MCV)', 'Eosinophils',
       'Mean corpuscular hemoglobin (MCH)', 'Basophils', 'Leukocytes',
       'Mean corpuscular hemoglobin concentration (MCHC)', 'Lymphocytes',
       'Red blood Cells', 'Monocytes', 'Mean platelet volume ', 'Neutrophils',
       'Proteina C reativa mg/dL', 'Creatinine', 'Urea', 'Potassium', 'Sodium',
       'Strepto A', 'Aspartate transaminase', 'Alanine transaminase',
       'Serum Glucose', 'Total Bilirubin', 'Direct Bilirubin',
       'Indirect Bilirubin', 'Gamma-glutamyltransferase ',
       'Alkaline phosphatase', 'HCO3 (venous blood gas analysis)',
       'pH (venous blood gas analysis)',
       'Total CO2 (venous blood gas analysis)',
       'Base excess (venous blood gas analysis)',
       'pO2 (venous blood gas analysis)', 'pCO2 (venous blood gas analysis)',
       'Hb sa

In [39]:
bloods = ['Proteina C reativa mg/dL', 
                'Neutrophils', 
                'Mean platelet volume ', 
                'Monocytes', 
                'Red blood cell distribution width (RDW)', 
                'Red blood Cells',
                'Platelets',
                'Eosinophils',
                'Basophils',
                'Leukocytes',
                'Mean corpuscular hemoglobin (MCH)',
                'Mean corpuscular volume (MCV)',
                'Mean corpuscular hemoglobin concentration\xa0(MCHC)',
                'Lymphocytes',
                'Hemoglobin',
                'Hematocrit']
influenza = ['Influenza A rapid test','Influenza B rapid test']

In [40]:
brazil_vars = ["SARS-Cov-2 exam result","Patient age quantile"] + bloods + influenza
brazil_vars

['SARS-Cov-2 exam result',
 'Patient age quantile',
 'Proteina C reativa mg/dL',
 'Neutrophils',
 'Mean platelet volume ',
 'Monocytes',
 'Red blood cell distribution width (RDW)',
 'Red blood Cells',
 'Platelets',
 'Eosinophils',
 'Basophils',
 'Leukocytes',
 'Mean corpuscular hemoglobin (MCH)',
 'Mean corpuscular volume (MCV)',
 'Mean corpuscular hemoglobin concentration\xa0(MCHC)',
 'Lymphocytes',
 'Hemoglobin',
 'Hematocrit',
 'Influenza A rapid test',
 'Influenza B rapid test']